## Required libraries

In [1]:
!pip install -U minsearch qdrant_client

  Using cached minsearch-0.0.4-py3-none-any.whl.metadata (8.1 kB)
Using cached minsearch-0.0.4-py3-none-any.whl (11 kB)
  Attempting uninstall: minsearch
    Found existing installation: minsearch 0.0.2
    Uninstalling minsearch-0.0.2:
      Successfully uninstalled minsearch-0.0.2
  Attempting uninstall: qdrant_client
    Found existing installation: qdrant-client 1.14.2
    Uninstalling qdrant-client-1.14.2:
      Successfully uninstalled qdrant-client-1.14.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [qdrant_client]m 1/2 [qdrant_client]


## Evaluation data

In [2]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [3]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

## Q1. Minsearch text

In [6]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [7]:
def minsearch_search(query, course):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [8]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hit_rate(relevance_total)

  0%|          | 0/4627 [00:00<?, ?it/s]

0.848714069591528

## Q2. Vector search for question

In [24]:
from minsearch import VectorSearch

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [26]:
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [27]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [28]:
def minsearch_vector_search(vector, course):
    return vindex.search(
        vector,
        filter_dict={'course': course},
        num_results=5
    )

def question_text_vector(q):
    question = q['question']
    course = q['course']

    v_q = pipeline.transform([question])

    return minsearch_vector_search(v_q, course)

In [29]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    # results = minsearch_vector_search(query=q['question'], course=q['course'])
    results = question_text_vector(dict(
    question=q['question'],
    course=q['course']
))
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

mrr(relevance_total)

  0%|          | 0/4627 [00:00<?, ?it/s]

0.3572833369353793

## Q3. Vector search for question and answer

In [30]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

In [33]:
X = pipeline.fit_transform(texts)
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [34]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    # results = minsearch_vector_search(query=q['question'], course=q['course'])
    results = question_text_vector(dict(
    question=q['question'],
    course=q['course']
))
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hit_rate(relevance_total)

  0%|          | 0/4627 [00:00<?, ?it/s]

0.8210503566025502

## Q4. Qdrant

In [35]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

In [39]:
collection_name = "ml-zoomcamp-faq"
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Create the collection with specified sparse vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [40]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [60]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [66]:
def vector_search(question,course):    
    # course = 'data-engineering-zoomcamp'
    query_points = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [65]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    # results = minsearch_vector_search(query=q['question'], course=q['course'])
    results = vector_search(
    question=q['question'],
    course=q['course']
)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

mrr(relevance_total)

  0%|          | 0/4627 [00:00<?, ?it/s]

0.68007708378359

## Q5. Cosine simiarity

In [82]:
import numpy as np
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

In [72]:
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

In [73]:
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

In [78]:
X=pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

In [80]:
X_llm = pipeline.transform(df_results.answer_llm)
X_orig = pipeline.transform(df_results.answer_orig)

In [86]:
cosine_similarities = [
    cosine(u, v) for u, v in zip(X_llm, X_orig)
]

float(df_results['cosine_similarity'].mean())

0.8415841233490402

## Q6. Rouge

In [87]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [92]:
from rouge import Rouge
rouge_scorer = Rouge()

# r = df_results.iloc[10]
# scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
# scores
rouge_1_f1_scores=[]
for _, row in tqdm(df_results.iterrows(), total=len(df_results)):
    try:
        score = rouge_scorer.get_scores(row.answer_llm, row.answer_orig)[0]
        rouge_1_f1_scores.append(score['rouge-1']['f'])
    except:
        rouge_1_f1_scores.append(0.0)  # fallback in case of error

avg_rouge_1_f1 = float(np.mean(rouge_1_f1_scores))
avg_rouge_1_f1

  0%|          | 0/1830 [00:00<?, ?it/s]

0.3516946452113943